In [1]:
import pandas as pd

For now, I only want to use public hospitals.

In [2]:
# import hospitals to dataframes
public = pd.read_csv('4_LegalEntitySummaryPublicHospital.csv')
#private = pd.read_csv('4_LegalEntitySummaryNGOHospital.csv')

In [3]:
# inspect both datasets
public.head()

,Premises Name,Certification Service Type,Service Types,Total Beds,Premises Website,Premises Address Other,Premises Address,Premises Address Suburb/Road,Premises Address Town/City,Premises Address Post Code,DHB Name,Certificate Name,Certification Period (Months),Certificate/License End Date,Current Auditor,Legal Name,Legal Entity Address Other,Legal Entity Address,Legal Entity Address Suburb/Road,Legal Entity Address Town/City,Legal Entity Address Post Code,Legal Entity Postal Address,Legal Entity Postal Address Suburb/Road,Legal Entity Postal Address Town/City,Legal Entity Postal Address Post Code,Legal Entity Website,Unnamed: 26
0,Auckland City Hospital,Public Hospital,"Childrens health, Maternity, Surgical, Medical",1165,NaN,NaN,2 Park Road,Grafton,Auckland,1023,Auckland District Health Board,Auckland District Health Board,No change,1-5-2023,The DAA Group Limited,Auckland District Health Board,"Level 51, Building 10, Greenlane Clinical Centre",214 Green Lane West,NaN,Auckland,1051,Private Bag 92024,NaN,Auckland,1142,http://www.adhb.govt.nz/,NaN
1,Auckland DHB X 3 Units - Mental Health,Public Hospital,Mental health,96,NaN,NaN,2 Park Road,Grafton,Auckland,1023,Auckland District Health Board,Auckland District Health Board,No change,1-5-2023,The DAA Group Limited,Auckland District Health Board,"Level 51, Building 10, Greenlane Clinical Centre",214 Green Lane West,NaN,Auckland,1051,Private Bag 92024,NaN,Auckland,1142,http://www.adhb.govt.nz/,NaN
2,Buchanan Rehabilitation Centre,Public Hospital,Mental health,40,NaN,NaN,27 Sutherland Road,Point Chevalier,Auckland,1025,Auckland District Health Board,Auckland District Health Board,No change,1-5-2023,The DAA Group Limited,Auckland District Health Board,"Level 51, Building 10, Greenlane Clinical Centre",214 Green Lane West,NaN,Auckland,1051,Private Bag 92024,NaN,Auckland,1142,http://www.adhb.govt.nz/,NaN
3,Greenlane Clinical Centre,Public Hospital,"Surgical, Medical",31,NaN,NaN,214 Green Lane West,Epsom,Auckland,1051,Auckland District Health Board,Auckland District Health Board,No change,1-5-2023,The DAA Group Limited,Auckland District Health Board,"Level 51, Building 10, Greenlane Clinical Centre",214 Green Lane West,NaN,Auckland,1051,Private Bag 92024,NaN,Auckland,1142,http://www.adhb.govt.nz/,NaN
4,Tauranga Hospital,Public Hospital,"Psychogeriatric, Geriatric, Mental health, Chi...",360,NaN,NaN,829 Cameron Road,Tauranga South,Tauranga,3112,Bay of Plenty District Health Board,Bay of Plenty District Health Board,36,3-10-2023,The DAA Group Limited,Bay of Plenty District Health Board,NaN,829 Cameron Road,NaN,Tauranga,3112,Private Bag 12024,NaN,Tauranga,3143,NaN,NaN


In [4]:
#private.head()

In [5]:
print(len(public))
#print(len(private))

83


There are 83 public hospitals and 72 private. Both dataset seem to have the same columns, hence I should be able to concatenate them into a single list.

In [6]:
#hospitals = pd.concat([public, private])
#print(len(hospitals))

As expected, the concatenated list contains 155 records.

Instead of the above, I will only use public hospitals.

In [12]:
hospitals = public.copy()

In [13]:
hospitals.head(1)

,Premises Name,Certification Service Type,Service Types,Total Beds,Premises Website,Premises Address Other,Premises Address,Premises Address Suburb/Road,Premises Address Town/City,Premises Address Post Code,DHB Name,Certificate Name,Certification Period (Months),Certificate/License End Date,Current Auditor,Legal Name,Legal Entity Address Other,Legal Entity Address,Legal Entity Address Suburb/Road,Legal Entity Address Town/City,Legal Entity Address Post Code,Legal Entity Postal Address,Legal Entity Postal Address Suburb/Road,Legal Entity Postal Address Town/City,Legal Entity Postal Address Post Code,Legal Entity Website,Unnamed: 26
0,Auckland City Hospital,Public Hospital,"Childrens health, Maternity, Surgical, Medical",1165,NaN,NaN,2 Park Road,Grafton,Auckland,1023,Auckland District Health Board,Auckland District Health Board,No change,1-5-2023,The DAA Group Limited,Auckland District Health Board,"Level 51, Building 10, Greenlane Clinical Centre",214 Green Lane West,NaN,Auckland,1051,Private Bag 92024,NaN,Auckland,1142,http://www.adhb.govt.nz/,NaN


In [14]:
# Next, I will only keep the relevant columns
hospitals = hospitals[['Premises Name', 'Certification Service Type', ' Premises Address',
                       ' Premises Address Suburb/Road', ' Premises Address Town/City', 
                       ' Premises Address Post Code', ' DHB Name']]
# combine premises address with suburb/road
hospitals[' Premises Address'] = hospitals[' Premises Address'] + ', ' +  hospitals[' Premises Address Suburb/Road']
# and change Subur/Road to City
hospitals[' Premises Address Suburb/Road'] = hospitals[' Premises Address Town/City']
hospitals[' Premises Address Town/City'] = ''
# this is so that the name of the state matches with the rest of the data
# and rename them to match the format
hospitals = hospitals.rename(columns={'Premises Name': 'name', ' Premises Address': 'address',
                                      ' Premises Address Suburb/Road': 'city',
                                      ' Premises Address Town/City': 'state',
                                      ' Premises Address Post Code': 'zip',
                                      'Certification Service Type': 'type',
                                      ' DHB Name': 'county'})
# and add the missing columns
hospitals[''] = hospitals.index
hospitals['id'] = hospitals.index
hospitals['phone'] = ''
hospitals['ownership'] = ''
hospitals['emergency'] = ''
hospitals['quality'] = ''
# and reaarange them
hospitals = hospitals[['id', 'name', 'address', 'city', 'state', 'zip', 'county',
                       'phone', 'type', 'ownership', 'emergency', 'quality']]
# check that I have what I expected
hospitals.head()

,id,name,address,city,state,zip,county,phone,type,ownership,emergency,quality
0,0,Auckland City Hospital,"2 Park Road, Grafton",Auckland,,1023,Auckland District Health Board,,Public Hospital,,,
1,1,Auckland DHB X 3 Units - Mental Health,"2 Park Road, Grafton",Auckland,,1023,Auckland District Health Board,,Public Hospital,,,
2,2,Buchanan Rehabilitation Centre,"27 Sutherland Road, Point Chevalier",Auckland,,1025,Auckland District Health Board,,Public Hospital,,,
3,3,Greenlane Clinical Centre,"214 Green Lane West, Epsom",Auckland,,1051,Auckland District Health Board,,Public Hospital,,,
4,4,Tauranga Hospital,"829 Cameron Road, Tauranga South",Tauranga,,3112,Bay of Plenty District Health Board,,Public Hospital,,,


Finally, I need to obtain state and coordinates for each hospital. I can use GeoPy again for this.

In [24]:
# install needed libraries
!pip install geopy 
!pip install Nominatim

  Created wheel for Nominatim: filename=nominatim-0.1-py3-none-any.whl size=2363 sha256=022133899e80632b71d7113026e2365856ec9f56e8dea019ceb32114ef2f01cf
  Stored in directory: /root/.cache/pip/wheels/37/00/9e/d904c390bfb174830ad3dcfd62af5544cee7d88bed4f8acedd
Successfully built Nominatim


In [25]:
# initiate geolocator
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Your_Name")

In [28]:
# set up a list of states
states = ['Northland', 'Auckland', 'Waikato', 'Manawatu-Wanganui',
       'Bay of Plenty', "Hawke's Bay", 'Gisborne', 'Taranaki',
       'Wellington', 'Area Outside', 'Tasman', 'Nelson', 'Marlborough',
       'Canterbury', 'West Coast', 'Otago', 'Southland']

In [26]:
# set up empty columns for latitude and longitude
hospitals['LAT'] = ''
hospitals['LON'] = ''

In [37]:
# iterate over all hospitals
for index, row in hospitals.iterrows():
    # get the location
    # I found that just looking by postcode produced the best result
    address = ' '.join([str(row['zip']), "New Zealand"])
    location = geolocator.geocode({address})
    # if location has been found
    if location != None:
      # add the coordinates to the dataframe
      hospitals.at[index,'LAT'] = location.latitude
      hospitals.at[index,'LON'] = location.longitude
      # iterate over each state and check if its name is in the location
      for state in states:
        if state in location.address:
          hospitals.at[index,'state'] = state

In [38]:
# check if all locations were found
hospitals.loc[hospitals['LAT'] == ''].count()

id           0
name         0
address      0
city         0
state        0
zip          0
county       0
phone        0
type         0
ownership    0
emergency    0
quality      0
LAT          0
LON          0
dtype: int64

In [39]:
# check if all states were found
hospitals.loc[hospitals['state'] == ''].count()

id           4
name         4
address      4
city         4
state        4
zip          4
county       4
phone        4
type         4
ownership    4
emergency    4
quality      4
LAT          4
LON          4
dtype: int64

The coordinates for all hospitals were count, however, the state for 4 of them were not. I can amend these manually.

In [40]:
for index, row in hospitals.iterrows():
  # if no coordinates have been found
  if row['state'] == '':
    print(f"The name of the hospital is: {row['name']}")
    address = ' '.join([str(row['address']), str(row['city']), str(row['zip']), "New Zealand"])
    print(f"The address is: {address}")
    state = input('State:')
    # add the found data to the relevant columns
    hospitals.at[index,'state'] = state

The name of the hospital is: Palmerston North Hospital
The address is: 50 Ruahine Street, Roslyn Palmerston North 4414 New Zealand
State:Manawatu-Wanganui
The name of the hospital is: Te Papaioea Birthing Centre
The address is: 117 Ruahine Street, Roslyn Palmerston North 4414 New Zealand
State:Manawatu-Wanganui
The name of the hospital is: Waikato Hospital
The address is: 183 Pembroke Street, Waikato Hospital Hamilton 3240 New Zealand
State:Waikato
The name of the hospital is: Whanganui Hospital
The address is: 100 Heads Road, Gonville Wanganui 4501 New Zealand
State:Manawatu-Wanganui


In [43]:
# check if all the regions are represented
len(hospitals['state'].unique())

15

Not all states are represented, let's check which are not.

In [44]:
list(set(states) - set(hospitals['state'].unique()))

['Northland', 'Area Outside']

Because there are only 83 hospitals, I will amend them manually in a spreadsheet.<br><br>
The issues with 'Northland' was that the postcodes start with 0 and are 4 digits long. Zero got removed and the postcode was pointing to the wrong location. This affected 5 hospitals. For ease, I manually amended these records in Excel.<br><br>
The 'Area Outside' does not have a hospital as it covers anything not covered elsewhere. I could either remove all areas that do not belong to a region or duplicate the records for one of the biggest hospitals in New Zealand and assign it to 'Area Outside', which is what I have done in Excel.

In [48]:
# I noticed that my index contains duplicates, I need to reset it.
hospitals = hospitals.reset_index()
hospitals['_'] = hospitals.index
hospitals['id'] = hospitals.index
hospitals = hospitals[['_', 'id', 'name', 'address', 'city', 'state', 'zip', 'county', 'phone',
       'type', 'ownership', 'emergency', 'quality', 'LAT', 'LON']]

## Save as csv


In [46]:
hospitals.to_csv('hospitals.csv', index=False)